In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import random
from tqdm import tqdm
from sklearn.metrics import balanced_accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import StandardScaler

# Set random seed for reproducibility
seed = 42
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

# Custom label smoothing loss with class weights
class LabelSmoothingCrossEntropyWeighted(nn.Module):
    def __init__(self, smoothing=0.1, class_weights=None):
        super().__init__()
        self.smoothing = smoothing
        self.class_weights = class_weights

    def forward(self, x, target):
        logprobs = nn.functional.log_softmax(x, dim=-1)
        nll_loss = -logprobs.gather(dim=-1, index=target.unsqueeze(1)).squeeze(1)
        smooth_loss = -logprobs.mean(dim=-1)
        if self.class_weights is not None:
            weights = self.class_weights[target]
            loss = weights * (1 - self.smoothing) * nll_loss + self.smoothing * smooth_loss
        else:
            loss = (1 - self.smoothing) * nll_loss + self.smoothing * smooth_loss
        return loss.mean()

# Neural network model
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, dropout_rate):
        super().__init__()
        layers = [nn.Linear(input_size, hidden_size), nn.SiLU(), nn.Dropout(dropout_rate)]
        for _ in range(num_layers - 1):
            layers += [nn.Linear(hidden_size, hidden_size), nn.SiLU(), nn.Dropout(dropout_rate)]
        layers.append(nn.Linear(hidden_size, output_size))
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x)

# Load and preprocess data
X = np.loadtxt("Xtr.csv", delimiter=",")
y = np.loadtxt("ytr.csv", delimiter=",")
scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=seed
)

X_train = torch.tensor(X_train, dtype=torch.float32)
X_val = torch.tensor(X_val, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
y_val = torch.tensor(y_val, dtype=torch.long)

# Compute class weights
class_weights = compute_class_weight('balanced', classes=np.unique(y), y=y)
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float32)

# Model configuration
input_size = X_train.shape[1]
hidden_size = 128
num_layers = 5
output_size = len(np.unique(y))
dropout_rate = 0.05
learning_rate = 0.001
weight_decay = 1e-4
num_epochs = 20000
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move data and model to device
X_train, X_val = X_train.to(device), X_val.to(device)
y_train, y_val = y_train.to(device), y_val.to(device)
model = NeuralNet(input_size, hidden_size, num_layers, output_size, dropout_rate).to(device)
criterion = LabelSmoothingCrossEntropyWeighted(smoothing=0.1, class_weights=class_weights_tensor.to(device))
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

# Train model
best_val_bacc = 0
best_epoch = 0
model_save_path = "best_model_dropout_005_l2.pth"

for epoch in tqdm(range(1, num_epochs + 1)):
    model.train()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    model.eval()
    with torch.no_grad():
        val_logits = model(X_val)
        val_preds = torch.argmax(val_logits, dim=1)
        val_bacc = balanced_accuracy_score(y_val.cpu().numpy(), val_preds.cpu().numpy())

        if val_bacc > best_val_bacc:
            best_val_bacc = val_bacc
            best_epoch = epoch
            torch.save(model.state_dict(), model_save_path)

        if epoch % 100 == 0:
            print(f"Epoch {epoch} | Val BACC: {val_bacc:.4f} | Best: {best_val_bacc:.4f} @ {best_epoch}")
            correct = (val_preds == y_val).cpu().numpy()
            for cls in np.unique(y):
                cls_mask = y_val.cpu().numpy() == cls
                cls_acc = correct[cls_mask].sum() / cls_mask.sum()
                print(f"Class {cls}: {cls_acc:.4f}", end=" | ")
            print()

print(f"Best Val Balanced Accuracy: {best_val_bacc:.4f} at epoch {best_epoch}")

# Save traced model on CPU
model.to("cpu")
x = torch.randn(1, input_size)  # shape: (1, 9)
with torch.no_grad():
    traced_model = torch.jit.trace(model, x)
torch.jit.save(traced_model, "model_neuralnet.pth")



In [ ]:
import torch
import numpy as np
from sklearn.preprocessing import StandardScaler

# Load TorchScript model (must be traced on CPU)
model = torch.jit.load("model_neuralnet.pth")
model.eval()

# Load data
Xka = np.loadtxt("Xka.csv", delimiter=",")
Xtr = np.loadtxt("Xtr.csv", delimiter=",")

# Standardize using Xtr
scaler = StandardScaler()
Xtr_std = scaler.fit_transform(Xtr)
Xka_std = scaler.transform(Xka)

# Predict on CPU
Xka_tensor = torch.tensor(Xka_std, dtype=torch.float32)
with torch.no_grad():
    yka_hat = model(Xka_tensor).argmax(dim=1).numpy()

# Save prediction
np.savetxt("yka_hat_neuralnet.csv", yka_hat, fmt="%d", delimiter=",")
